https://datahack.analyticsvidhya.com/contest/job-a-thon-august-2022/#LeaderBoard

🔥🔥 122 position out of 959 participants 🔥🔥

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train =  pd.read_csv('../input/jobathon-data/train_F3fUq2S.csv')
test = pd.read_csv('../input/jobathon-data/test_Bk2wfZ3.csv')

In [3]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
#             print("******************************")
#             print("Column: ",col)
#             print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
#             print("dtype after: ",props[col].dtype)
#             print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist
train, NAlist = reduce_mem_usage(train)
test, NAlist = reduce_mem_usage(test)

In [9]:
train1 = train.drop(['campaign_id','is_price','is_timer'],axis=1)
test1 = test.drop(['campaign_id','is_price','is_timer'],axis=1)

In [10]:
def encoder(col,train2,test2):
    import random
    import string
    from sklearn.preprocessing import OneHotEncoder
    enc = OneHotEncoder(handle_unknown='ignore')
    transformed_train = pd.DataFrame(enc.fit_transform(train2[[col]]).toarray())
    transformed_test = pd.DataFrame(enc.transform(test2[[col]]).toarray())
    
    a_train = train2[col].value_counts()
    a_test = test2[col].value_counts()
    N = 5
    
    if(transformed_train.shape[1]<=a_train.count()):
        u = a_train.keys()[range(0,len(a_train.keys()))].tolist()
        for i,j in enumerate(u):
            u[i] = str(j)+'_'+col
    else:
        u = a_test.keys()[range(0,len(a_test.keys()))].tolist()
        for i,j in enumerate(u):
            u[i] = str(j)+'_'+col
        bz = transformed_train.shape[1]-a_train.count()
        for i in range(bz):
            u.append(''.join(random.choices(string.ascii_uppercase +
                             string.digits, k = N)))
    transformed_train.columns = u
    transformed_test.columns  = u
    train2.drop(col, axis=1, inplace = True)
    test2.drop(col, axis=1, inplace = True)
    transformed_train.index = train2.index
    transformed_test.index = test2.index
    train2 = train2.join(transformed_train)
    test2 = test2.join(transformed_test)
    return(train2, test2)

In [11]:
train2, test2 = encoder('times_of_day',train1,test1)

In [12]:
train3 = train2.drop('Morning_times_of_day', axis=1)
test3 = test2.drop('Morning_times_of_day', axis=1)

In [14]:
train_11 = train3.copy()
test_11 = test3.copy()

In [15]:
train_11.loc[(train_11['no_of_CTA'] != 0),'mean_CTA_len_div_no_of_CTA'] = train_11['mean_CTA_len']/train_11['no_of_CTA']
train_11.loc[(train_11['target_audience'] != 0),'body_len_div_target_audience'] = train_11['body_len']/train_11['target_audience']
train_11.loc[(train_11['target_audience'] != 0),'no_of_CTA_div_target_audience'] = train_11['no_of_CTA']/train_11['target_audience']
train_11.loc[(train_11['body_len'] != 0),'target_audience_div_body_len'] = train_11['target_audience']/train_11['body_len']
train_11.loc[(train_11['mean_CTA_len'] != 0),'product_div_mean_CTA_len'] = train_11['product']/train_11['mean_CTA_len']
train_11.loc[(train_11['category'] != 0),'mean_paragraph_len_div_category'] = train_11['mean_paragraph_len']/train_11['category']
train_11.loc[(train_11['no_of_CTA'] != 0),'target_audience_div_no_of_CTA '] = train_11['target_audience']/train_11['no_of_CTA']
train_11.loc[(train_11['body_len'] != 0),'no_of_CTA_div_body_len'] = train_11['no_of_CTA']/train_11['no_of_CTA']
train_11.loc[(train_11['body_len'] != 0),'body_len_log_mul'] = train_11['body_len']*np.log(train_11['body_len'])
train_11.loc[(train_11['subject_len'] != 0),'target_audience_div_subject_len'] = train_11['target_audience']/train_11['subject_len']
train_11.loc[(train_11['no_of_CTA'] != 0),'body_len_div_no_of_CTA'] = train_11['body_len']/train_11['no_of_CTA']
train_11.loc[(train_11['product'] != 0),'body_len_div_product'] = train_11['body_len']/train_11['product']
train_11.loc[(train_11['product'] != 0),'no_of_CTA_div_product'] = train_11['no_of_CTA']/train_11['product']

train_11.loc[(train_11['no_of_CTA'] == 0),'mean_CTA_len_div_no_of_CTA'] = train_11['mean_CTA_len']/(train_11['no_of_CTA']+1)
train_11.loc[(train_11['target_audience'] == 0),'body_len_div_target_audience'] = train_11['body_len']/(train_11['target_audience']+1)
train_11.loc[(train_11['target_audience'] == 0),'no_of_CTA_div_target_audience'] = train_11['no_of_CTA']/(train_11['target_audience']+1)
train_11.loc[(train_11['body_len'] == 0),'target_audience_div_body_len'] = train_11['target_audience']/(train_11['body_len']+1)
train_11.loc[(train_11['mean_CTA_len'] == 0),'product_div_mean_CTA_len'] = train_11['product']/(train_11['mean_CTA_len']+1)
train_11.loc[(train_11['category'] == 0),'mean_paragraph_len_div_category'] = train_11['mean_paragraph_len']/(train_11['category']+1)
train_11.loc[(train_11['no_of_CTA'] == 0),'target_audience_div_no_of_CTA '] = train_11['target_audience']/(train_11['no_of_CTA']+1)
train_11.loc[(train_11['body_len'] == 0),'no_of_CTA_div_body_len'] = train_11['no_of_CTA']/(train_11['no_of_CTA']+1)
train_11.loc[(train_11['body_len'] == 0),'body_len_log_mul'] = train_11['body_len']*(np.log(train_11['body_len'])+1)
train_11.loc[(train_11['subject_len'] == 0),'target_audience_div_subject_len'] = train_11['target_audience']/(train_11['subject_len']+1)
train_11.loc[(train_11['no_of_CTA'] == 0),'body_len_div_no_of_CTA'] = train_11['body_len']/(train_11['no_of_CTA']+1)
train_11.loc[(train_11['product'] == 0),'body_len_div_product'] = train_11['body_len']/(train_11['product']+1)
train_11.loc[(train_11['product'] == 0),'no_of_CTA_div_product'] = train_11['no_of_CTA']/(train_11['product']+1)

In [16]:
test_11.loc[(test_11['no_of_CTA'] != 0),'mean_CTA_len_div_no_of_CTA'] = test_11['mean_CTA_len']/test_11['no_of_CTA']
test_11.loc[(test_11['target_audience'] != 0),'body_len_div_target_audience'] = test_11['body_len']/test_11['target_audience']
test_11.loc[(test_11['target_audience'] != 0),'no_of_CTA_div_target_audience'] = test_11['no_of_CTA']/test_11['target_audience']
test_11.loc[(test_11['body_len'] != 0),'target_audience_div_body_len'] = test_11['target_audience']/test_11['body_len']
test_11.loc[(test_11['mean_CTA_len'] != 0),'product_div_mean_CTA_len'] = test_11['product']/test_11['mean_CTA_len']
test_11.loc[(test_11['category'] != 0),'mean_paragraph_len_div_category'] = test_11['mean_paragraph_len']/test_11['category']
test_11.loc[(test_11['no_of_CTA'] != 0),'target_audience_div_no_of_CTA '] = test_11['target_audience']/test_11['no_of_CTA']
test_11.loc[(test_11['body_len'] != 0),'no_of_CTA_div_body_len'] = test_11['no_of_CTA']/test_11['no_of_CTA']
test_11.loc[(test_11['body_len'] != 0),'body_len_log_mul'] = test_11['body_len']*np.log(test_11['body_len'])
test_11.loc[(test_11['subject_len'] != 0),'target_audience_div_subject_len'] = test_11['target_audience']/test_11['subject_len']
test_11.loc[(test_11['no_of_CTA'] != 0),'body_len_div_no_of_CTA'] = test_11['body_len']/test_11['no_of_CTA']
test_11.loc[(test_11['product'] != 0),'body_len_div_product'] = test_11['body_len']/test_11['product']
test_11.loc[(test_11['product'] != 0),'no_of_CTA_div_product'] = test_11['no_of_CTA']/test_11['product']

test_11.loc[(test_11['no_of_CTA'] == 0),'mean_CTA_len_div_no_of_CTA'] = test_11['mean_CTA_len']/(test_11['no_of_CTA']+1)
test_11.loc[(test_11['target_audience'] == 0),'body_len_div_target_audience'] = test_11['body_len']/(test_11['target_audience']+1)
test_11.loc[(test_11['target_audience'] == 0),'no_of_CTA_div_target_audience'] = test_11['no_of_CTA']/(test_11['target_audience']+1)
test_11.loc[(test_11['body_len'] == 0),'target_audience_div_body_len'] = test_11['target_audience']/(test_11['body_len']+1)
test_11.loc[(test_11['mean_CTA_len'] == 0),'product_div_mean_CTA_len'] = test_11['product']/(test_11['mean_CTA_len']+1)
test_11.loc[(test_11['category'] == 0),'mean_paragraph_len_div_category'] = test_11['mean_paragraph_len']/(test_11['category']+1)
test_11.loc[(test_11['no_of_CTA'] == 0),'target_audience_div_no_of_CTA '] = test_11['target_audience']/(test_11['no_of_CTA']+1)
test_11.loc[(test_11['body_len'] == 0),'no_of_CTA_div_body_len'] = test_11['no_of_CTA']/(test_11['no_of_CTA']+1)
test_11.loc[(test_11['body_len'] == 0),'body_len_log_mul'] = test_11['body_len']*(np.log(test_11['body_len'])+1)
test_11.loc[(test_11['subject_len'] == 0),'target_audience_div_subject_len'] = test_11['target_audience']/(test_11['subject_len']+1)
test_11.loc[(test_11['no_of_CTA'] == 0),'body_len_div_no_of_CTA'] = test_11['body_len']/(test_11['no_of_CTA']+1)
test_11.loc[(test_11['product'] == 0),'body_len_div_product'] = test_11['body_len']/(test_11['product']+1)
test_11.loc[(test_11['product'] == 0),'no_of_CTA_div_product'] = test_11['no_of_CTA']/(test_11['product']+1)

In [19]:
target = 'click_rate'
X = train_11.drop(target,axis=1)
y = train_11[target]

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)
import catboost as cb
cat_features = []
train_dataset = cb.Pool(X_train, y_train,cat_features=cat_features) 
test_dataset = cb.Pool(X_test, y_test,cat_features=cat_features)
cat_model = cb.CatBoostRegressor(loss_function='RMSE',iterations=5000 )
cat_model.fit( X_train, y_train,
               eval_set=(X_test, y_test),
               cat_features=cat_features,
               plot=True
              )

In [21]:
import math
from sklearn.metrics import mean_squared_error, r2_score
y_predict= cat_model.predict(X_test)
#RMSE
Rmse_test = math.sqrt(mean_squared_error(y_test,y_predict))
#R2 Score
r2_test = r2_score(y_test,y_predict)
# Adjusted R2 Score
n= X_train.shape[0] # total no of datapoints
p= X_train.shape[1] # total no of independent features
adj_r2_test = 1-(1-r2_test)*(n-1)/(n-p-1)
#print results
print("Evaluation on test data")
print("RMSE: {:.2f}".format(Rmse_test))
print("R2: {:.2f}".format(r2_test))
print("Adjusted R2: {:.2f}".format(adj_r2_test))

In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)


X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.1, random_state=RANDOM_SEED)



In [28]:
def objective(trial):
    param = {}
    param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.02, 0.001)
    param['depth'] = trial.suggest_int('depth', 9, 15)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 10000
    param['use_best_model'] = True
    param['eval_metric'] = 'RMSE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = RANDOM_SEED
    param['logging_level'] = 'Silent'
    
    regressor = CatBoostRegressor(**param)

    regressor.fit(X_train.copy(), y_train.copy(),
                  eval_set=[(X_eval.copy(), y_eval.copy())],
                  early_stopping_rounds=EARLY_STOPPING_ROUND)
    loss = mean_squared_error(y_valid, regressor.predict(X_valid.copy()))
    return loss

In [34]:
%%time
import optuna


SAMPLE_RATE = 0.4
RANDOM_SEED = 1
EARLY_STOPPING_ROUND = 100
from catboost import CatBoostRegressor

study = optuna.create_study(study_name=f'catboost-seed{RANDOM_SEED}')
study.optimize(objective, n_trials=4000, n_jobs=-1, timeout=600)

In [35]:


study.best_value



In [36]:
study.best_params

In [37]:
%%time
optimized_regressor = CatBoostRegressor(learning_rate=study.best_params['learning_rate'],
                                        depth=study.best_params['depth'],
                                        l2_leaf_reg=study.best_params['l2_leaf_reg'],
                                        min_child_samples=study.best_params['min_child_samples'],
                                        grow_policy='Depthwise',
                                        iterations=10000,
                                        use_best_model=True,
                                        eval_metric='RMSE',
                                        od_type='iter',
                                        od_wait=20,
                                        random_state=RANDOM_SEED,
                                        logging_level='Silent')
optimized_regressor.fit(X_train.copy(), y_train.copy(),
                        eval_set=[(X_eval.copy(), y_eval.copy())],
                        early_stopping_rounds=EARLY_STOPPING_ROUND)
pred_train = optimized_regressor.predict(X_train.copy())
pred_valid = optimized_regressor.predict(X_valid.copy())

In [38]:
r2_score(y_train, pred_train)

In [39]:
r2_score(y_valid, pred_valid)

In [42]:
optimized_regressor.predict(test_11).shape

In [43]:
click_rate = optimized_regressor.predict(test_11)
submit = pd.DataFrame()
submit['campaign_id'] = test['campaign_id']
submit['click_rate'] = click_rate
submit.reset_index(drop=True, inplace=True)
submit.to_csv('submission3.csv', index = False)